AI PROJECT 2
TEXT SUMMARIZATION

In [1]:
#STEP 1 - IMPORT ALL THE REQUIRED LIBRARIES

In [2]:
!pip install transformers   #download and install the latest version of the transformers package
!pip install PyPDF2         # download and install the PyPDF2 library, which provides functionalities to work with PDF files in Python
!pip install rouge        #download and install the rouge library, which is used for evaluating the quality of text summaries
import PyPDF2         #imports the PyPDF2 module, which is commonly used for working with PDF files in Python
import torch             # import the PyTorch library to use its functionalities for tasks such as tensor operations, neural network building, and training
from transformers import AutoTokenizer, AutoModelWithLMHead   # imports the AutoTokenizer and AutoModelWithLMHead classes from the transformers library
from transformers import PegasusForConditionalGeneration, PegasusTokenizer  #imports the PegasusForConditionalGeneration and PegasusTokenizer classes from the transformers library for working with the Pegasus model
from transformers import BartTokenizer, BartForConditionalGeneration # imports the BartTokenizer and BartForConditionalGeneration classes from the transformers library for working with the BART
from rouge import Rouge   #imports the Rouge class from the rouge library

In [3]:
#STEP 2 - EXTRACT THE TEXT FROM ARTICLE WHICH IS IN PDF FORMART

In [4]:
def extract_text_from_pdf(pdf_path):  # function declaration
    text = ""   # initialize a variable as an empty string to store the text
    with open(pdf_path, "rb") as file:  #to open a PDF file in binary mode using a context manager
        reader = PyPDF2.PdfReader(file) #creating a object to read the PDF
        num_pages = len(reader.pages) # to calculate length of pdf
        for page_num in range(num_pages): # to iterate over each page
            page = reader.pages[page_num] # accessing a specific page from the PDF file
            text += page.extract_text() #store text that object reads
    return text # returns the extracted text of pdf

In [5]:
pdf_path = r"/content/PIIS1474442224000383.pdf"  #path of pdf
extracted_text = extract_text_from_pdf(pdf_path) #calling function to extract text
print(extracted_text)  # to print the extracted text

344  www.thelancet.com/neurology   Vol 23   April 2024
ArticlesGlobal, regional, and national burden of disorders affecting 
the nervous system, 1990–2021: a systematic analysis for the Global Burden of Disease Study 2021
GBD 2021 Nervous System Disorders Collaborators*
Summary
Background Disorders affecting the nervous system are diverse and include neurodevelopmental disorders, late-life 
neurodegeneration, and newly emergent conditions, such as cognitive impairment following COVID-19. Previous publications from the Global Burden of Disease, Injuries, and Risk Factor Study estimated the burden of 15 neurological conditions in 2015 and 2016, but these analyses did not include neurodevelopmental disorders, as defined by the International Classification of Diseases (ICD)-11, or a subset of cases of congenital, neonatal, and infectious conditions that cause neurological damage. Here, we estimate nervous system health loss caused by 37 unique conditions and their associated risk factors g

In [6]:
#STEP 3 - DEFINE ALL THE THREE MODELS FOR TEXT SUMMARIZATION I.E., T5, BART AND PEGASUS

T5 MODEL

In [7]:
# Load pre-trained T5 tokenizer and model
#Initialize the tokenizer and model outside the function for reusability
tokenizer = AutoTokenizer.from_pretrained('t5-base') #initialize the tokenizer with t5-base means use the base version of t5
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True) #initialize the model with t5-base means use the base version of t5

def t5_summarizer(extracted_text, max_length=200, min_length=80, length_penalty=5., num_beams=2):   #define a function named t5_summarizer that takes extracted_text as input along with parameters max_length, min_length, length_penalty, and num_beams, and returns a summary using the T5 model
    inputs = tokenizer.encode("summarize: " + extracted_text  #  tokenize the input text for summarization into token IDs, which can be processed by the T5 model
                              ,return_tensors='pt'  #means method should return PyTorch tensors
                              ,max_length=512  #specifies the maximum length of the tokenized sequence
                              ,truncation=True) # indicates that the input sequence should be truncated if its length exceeds the max_length

    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams) # generates a summary using the T5 model
    t5_summary = tokenizer.decode(summary_ids[0])  #decodes the token IDs of the generated summary back into text using the tokenizer
    return t5_summary #returns the summary generated by t5 summary

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1595: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


BART

In [22]:
# Load pre-trained BART tokenizer and model
#Initialize the tokenizer and model outside the function for reusability
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')  #initialize the tokenizer with the pre-trained weights from the facebook/bart-large-cnn
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')  #initialize the model with the pre-trained weights from the facebook/bart-large-cnn

def bart_summarize_text(text, max_length=1024, min_length=40):  #function definition
    # Tokenize input text
    inputs = tokenizer([text], max_length=512, return_tensors='pt', truncation=True)

    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, num_beams=4, early_stopping=True)

    # Decode and return the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return summary

Pegasus

In [14]:
# Load the pre-trained Pegasus model and tokenizer
model_name = "google/pegasus-xsum"  # Pegasus model fine-tuned on extreme summarization task
tokenizer = PegasusTokenizer.from_pretrained(model_name) #initialize the tokenizer with the pre-trained weights
model = PegasusForConditionalGeneration.from_pretrained(model_name) #initialize the model with the pre-trained weights

def pegasus_summarize_text(input_text, max_length=150): # function definition
    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)

    # Split the input into smaller segments if it exceeds the maximum token length
    segment_length = 512
    segments = [input_ids[:, i:i+segment_length] for i in range(0, input_ids.size(1), segment_length)]

    # Generate summaries for each segment
    summaries = []
    for segment in segments:
        summary_ids = model.generate(segment, max_length=max_length, num_beams=4, length_penalty=2.0, early_stopping=True)
        summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary_text)

    # Combine the summaries from all segments
    summary = " ".join(summaries)

    return summary


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
#STEP 4 - CALL ALL THE MODELS AND FIND SUMMARY FOR EACH MODEL

In [23]:
t5_summary = t5_summarizer(extracted_text) #calling the function
print(t5_summary) # Print the summary

</s><s>Diseases affecting the nervous system are diverse and include neurodevelopmental disorders, late-life neurodegeneration, and newly emergent conditions, such as cognitive impairment following COVID-19. We estimate nervous system health loss caused by 37 unique conditions and their associated risk factors globally, regionally, and nationally from 1990 to 2021. Globally, the 37 conditions affecting the nervous system were collectively ranked as the leading group.</s>


In [24]:
bart_summary = bart_summarize_text(extracted_text) #calling the function
print(bart_summary) # Print the summary

We estimate nervous system health loss caused by 37 unique conditions and their associated risk factors globally, regionally, and nationally from 1990 to 2021. We included morbidity and deaths due to neurological conditions, for which health loss is directly due to damage to the CNS or peripheral nervous system. Globally, the 37 conditions affecting the nervous system were collectively ranked as the leading group cause of death.


In [20]:
pegasus_summary = pegasus_summarize_text(extracted_text) #calling the function
print(pegasus_summary)  # Print the summary

The global burden of disorders affecting the nervous system has increased by 18% between 1990 and 2021 to 40 billion individuals, affecting 40% of the global population, according to a systematic analysis. The World Health Organization (WHO) and the International Agency for Research on Cancer (IARC) have published global data on disease burden of disorders affecting the nervous system (DALYs) for the period 1990 to 2021.


In [25]:
#STEP 5 - INITIALIZE THE ORIGINAL SUMMARY OF THE DOCUMENT

In [26]:
original_summary = "Disorders affecting the nervous system are diverse and include neurodevelopmental disorders, late-life neurodegeneration, and newly emergent conditions, such as cognitive impairment following COVID-19. Previous publications from the Global Burden of Disease, Injuries, and Risk Factor Study estimated the burden of 15 neurological conditions in 2015 and 2016, but these analyses did not include neurodevelopmental disorders, as defined by the International Classification of Diseases (ICD)-11, or a subset of cases of congenital, neonatal, and infectious conditions that cause neurological damage. Here, we estimate nervous system health loss caused by 37 unique conditions and their associated risk factors globally, regionally, and nationally from 1990 to 2021"

In [27]:
#STEP 6 - CALCULATE THE ROUGE SCORES FOR EACH MODEL

In [28]:
# Initialize Rouge
rouge = Rouge()

# Calculate ROUGE scores for each agent
scores_t5 = rouge.get_scores(t5_summary, original_summary)
scores_bart = rouge.get_scores(bart_summary, original_summary)
scores_pegasus = rouge.get_scores(pegasus_summary, original_summary)

# Print ROUGE scores
print("ROUGE scores for T5:", scores_t5)
print("ROUGE scores for BART:", scores_bart)
print("ROUGE scores for Pegasus:", scores_pegasus)


ROUGE scores for T5: [{'rouge-1': {'r': 0.5, 'p': 0.8163265306122449, 'f': 0.6201550340484346}, 'rouge-2': {'r': 0.42, 'p': 0.7241379310344828, 'f': 0.531645564973562}, 'rouge-l': {'r': 0.5, 'p': 0.8163265306122449, 'f': 0.6201550340484346}}]
ROUGE scores for BART: [{'rouge-1': {'r': 0.3875, 'p': 0.6326530612244898, 'f': 0.4806201503275045}, 'rouge-2': {'r': 0.24, 'p': 0.3870967741935484, 'f': 0.29629629157140686}, 'rouge-l': {'r': 0.35, 'p': 0.5714285714285714, 'f': 0.4341085224205276}}]
ROUGE scores for Pegasus: [{'rouge-1': {'r': 0.1625, 'p': 0.29545454545454547, 'f': 0.20967741477627483}, 'rouge-2': {'r': 0.07, 'p': 0.11864406779661017, 'f': 0.08805030979787216}, 'rouge-l': {'r': 0.15, 'p': 0.2727272727272727, 'f': 0.19354838251821027}}]
